<a href="https://colab.research.google.com/github/mguelgui/picProcessCNN/blob/main/CNN_Part3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#################################################################
### Schritt 1: a) Embeddings extrahieren                      ###
###            b) Ähnlichkeit zwischen Vogelbildern berechnen ###
###            c) Ergebnisse in Ähnlichkeitsmatrix speichern  ###
#################################################################

import os                                               # (kann man weglassen, wenn alles auf einmal gerunt wird)
import tensorflow as tf
from tensorflow.keras.preprocessing import image        # image-Modul enthält Funktionen zur Vorverarbeitung von Bildern zB. in ein CNN-taugliches Format
from sklearn.metrics.pairwise import cosine_similarity  # Zum Berechnen der Kosinus-Ähnlichkeit
import numpy as np                                      # Zur Arbeit mit Arrays/Matrizen etc.
import pandas as pd                                     # Zum Export von Daten in CSV-Dateien

ordner_pfad = '/content/Tierbilder' # Pfad zum Ordner festgelegt, in dem sich die Bilder befinden
tierbilder = os.listdir(ordner_pfad) # Listet alle Dateien im Ordner ordner_pfad auf und speichert sie in der Liste "tierbilder"


### Funktion zum Extrahieren von Features mit mehreren vortrainierten Keras-Modellen ###

def extract_features_keras(model, preprocess_input, target_size, ordner_pfad):
    # Definiert eine Funktion `extract_features_keras`, die die Merkmale von Bildern in einem Ordner extrahiert.
    # Die Funktion nimmt vier Argumente entgegen: `model` (ein vortrainiertes Keras-Modell), `preprocess_input` (eine Funktion zur Bildvorverarbeitung),
    # `target_size` (die Zielgröße für die Bildskalierung, z. B. (224, 224)) und `ordner_pfad` (der Pfad zum Ordner mit den Bildern)
    dateien = sorted(os.listdir(ordner_pfad))     # Liest die Liste der Dateien im angegebenen `ordner_pfad` und sortiert sie alphabetisch
    features = []
    for datei in dateien:
        bild_pfad = os.path.join(ordner_pfad, datei)
        img = image.load_img(bild_pfad, target_size=target_size)
        img_array = image.img_to_array(img)
        img_array = preprocess_input(img_array)
        img_array = tf.expand_dims(img_array, axis=0)
        features.append(model.predict(img_array))
    return np.concatenate(features, axis=0)
# Bilder werden geladen, skaliert, in numpy-arrays umgewandelt; mit preprocess.input normalisiert (= an jeweilige CNN angepasst)
# Batch-Dimension wird mit tf.expand_dims zu jedem Bild hinzugefügt, damit jedes einzeln als Batch behandelt wird und nicht alle zusammen
# Durchlauf durch die verschiedenen Modelle und Rückgabe der Vektoren


### Erstellen eines Dictionarys aus Keras Modellen und deren Preprocessing-Funktionen etc. ###

keras_models = {
    'ResNet50': (tf.keras.applications.ResNet50(weights='imagenet', include_top=False, pooling='avg'),
                 tf.keras.applications.resnet50.preprocess_input, (224, 224)),
    'VGG16': (tf.keras.applications.VGG16(weights='imagenet', include_top=False, pooling='avg'),
              tf.keras.applications.vgg16.preprocess_input, (224, 224)),
    'InceptionV3': (tf.keras.applications.InceptionV3(weights='imagenet', include_top=False, pooling='avg'),
                    tf.keras.applications.inception_v3.preprocess_input, (299, 299)),
    'DenseNet121': (tf.keras.applications.DenseNet121(weights='imagenet', include_top=False, pooling='avg'),
                    tf.keras.applications.densenet.preprocess_input, (224, 224))
}
# Es wird jeweils das vortrainierte Modell mit den imagenet-Gewichten geladen;
# include_top=False bedeutet, dass die oberen Schichten (Klassifikationsschicht) entfernt werden, sodass nur die Convolutional-Layer (Merkmalextraktionsschichten) übrig bleiben
# pooling='avg' fügt eine durchschnittliche Pooling-Schicht hinzu, um den Ausgabe-Feature-Vektor zu reduzieren
# Danach kommt die spezifische Vorverarbeitungsfunktion und die Zielgröße der Bilder


### Extrahieren und speichern der Feature-Embeddings und Erstellen von Ähnlichkeitsmatrizen ###

# Iteration über das Dictionary mit der zuvor definierten Funktion zur Feature-Extraktion
for model_name, (model, preprocess_input, target_size) in keras_models.items():
    features = extract_features_keras(model, preprocess_input, target_size, ordner_pfad)

    # Export der Feature-Embeddings in eine CSV-Datei
    features_df = pd.DataFrame(features)
    features_df.to_csv(f'embeddings_{model_name}_2.csv', index=False)

    # Berechnen der Ähnlichkeitsmatrix (Berechnet für jeden Merkmalsvektor im Array features die Kosinus-Ähnlichkeit zu jedem anderen Vektor)
    similarity_matrix_2 = cosine_similarity(features)

    # Export der Ähnlichkeitsmatrix in eine CSV-Datei
    df = pd.DataFrame(similarity_matrix_2)
    df.to_csv(f'similarity_matrix_{model_name}_2.csv', index=False)

    print(f"{model_name} similarity matrix:") # Kann man weglassen, ist nur zum Prüfen des aktuellen Stands und Anschauen der Matrizen
    print(similarity_matrix_2)


### ACHTUNG: Hier wird Kosine-Korrelation verwendet, um zu beurteilen, wie ähnlich zwei Bilder im Merkmalsraum sind, unabhängig von der absoluten Größe der Merkmalsvektoren.


#####################################################################
### Schritt 2: Korrelieren der Ähnlichkeitsmatrizen untereinander ###
#####################################################################

# Wieder Dictionary erstellen zum Zugreifen auf die CSVs
similarity_matrices = {
    'ResNet50': pd.read_csv('similarity_matrix_ResNet50_2.csv'),
    'VGG16': pd.read_csv('similarity_matrix_VGG16_2.csv'),
    'InceptionV3': pd.read_csv('similarity_matrix_InceptionV3_2.csv'),
    'DenseNet121': pd.read_csv('similarity_matrix_DenseNet121_2.csv')
}

# Korrelation der Ähnlichkeitsmatrizen berechnen
correlations = {}
for name1, mat1 in similarity_matrices.items():
    for name2, mat2 in similarity_matrices.items():
        if name1 < name2:                     # Sicherstellen, dass jedes Modellpaar nur einmal betrachtet wird
            mat2 = mat2.set_index(mat1.index) # Sicherstellen, dass die beiden Ähnlichkeitsmatrizen mat1 und mat2 denselben Index haben
            corr = mat1.stack().corr(mat2.stack())
            correlations[f'{name1} vs {name2}'] = corr

# Ausgabe der Korrelationen
for comparison, correlation in correlations.items():
    print(f"Die Korrelation zwischen {comparison} beträgt: {correlation:.4f}")

# Exportiere die Korrelationsdaten
correlations_df = pd.DataFrame(list(correlations.items()), columns=['Vergleich', 'Korrelation'])
correlations_df.to_csv('aehnlichkeitsmatrix_korrelationen_2.csv', index=False)

### ACHTUNG: Es wird Pearson-Korrelation verwendet, um die Matrizen zu korrelieren
### (Es wird bewertet, ob hohe (bzw. niedrige) Kosinus-Ähnlichkeitswerte in einer Matrix auch hohe (bzw. niedrige) Werte in der anderen Matrix haben)


#################################################################
### Schritt 2: a) Lineare Dimensionsreduktion mit PCA         ###
###            b) Ähnlichkeit zwischen Vogelbildern berechnen ###
###            c) Ergebnisse in Ähnlichkeitsmatrix speichern  ###
#################################################################

import os                                               # (kann man weglassen, wenn alles auf einmal gerunt wird)
import tensorflow as tf
from tensorflow.keras.preprocessing import image        # image-Modul enthält Funktionen zur Vorverarbeitung von Bildern zB. in ein CNN-taugliches Format
from sklearn.metrics.pairwise import cosine_similarity  # Zum Berechnen der Kosinus-Ähnlichkeit
import numpy as np                                      # Zur Arbeit mit Arrays/Matrizen etc.
import pandas as pd                                     # Zum Export von Daten in CSV-Dateien
from sklearn.decomposition import PCA                   # PCA für die Dimensionsreduktion importieren

ordner_pfad = '/content/Tierbilder' # Pfad zum Ordner festgelegt, in dem sich die Bilder befinden
tierbilder = os.listdir(ordner_pfad) # Listet alle Dateien im Ordner ordner_pfad auf und speichert sie in der Liste "tierbilder"


### Funktion zum Extrahieren von Features mit mehreren vortrainierten Keras-Modellen ###

def extract_features_keras(model, preprocess_input, target_size, ordner_pfad):
    # Definiert eine Funktion `extract_features_keras`, die die Merkmale von Bildern in einem Ordner extrahiert.
    # Die Funktion nimmt vier Argumente entgegen: `model` (ein vortrainiertes Keras-Modell), `preprocess_input` (eine Funktion zur Bildvorverarbeitung),
    # `target_size` (die Zielgröße für die Bildskalierung, z. B. (224, 224)) und `ordner_pfad` (der Pfad zum Ordner mit den Bildern)
    dateien = sorted(os.listdir(ordner_pfad))     # Liest die Liste der Dateien im angegebenen `ordner_pfad` und sortiert sie alphabetisch
    features = []
    for datei in dateien:
        bild_pfad = os.path.join(ordner_pfad, datei)
        img = image.load_img(bild_pfad, target_size=target_size)
        img_array = image.img_to_array(img)
        img_array = preprocess_input(img_array)
        img_array = tf.expand_dims(img_array, axis=0)
        features.append(model.predict(img_array))
    return np.concatenate(features, axis=0)
# Bilder werden geladen, skaliert, in numpy-arrays umgewandelt; mit preprocess.input normalisiert (= an jeweilige CNN angepasst)
# Batch-Dimension wird mit tf.expand_dims zu jedem Bild hinzugefügt, damit jedes einzeln als Batch behandelt wird und nicht alle zusammen
# Durchlauf durch die verschiedenen Modelle und Rückgabe der Vektoren


### Erstellen eines Dictionarys aus Keras Modellen und deren Preprocessing-Funktionen etc. ###

keras_models = {
    'ResNet50': (tf.keras.applications.ResNet50(weights='imagenet', include_top=False, pooling='avg'),
                 tf.keras.applications.resnet50.preprocess_input, (224, 224)),
    'VGG16': (tf.keras.applications.VGG16(weights='imagenet', include_top=False, pooling='avg'),
              tf.keras.applications.vgg16.preprocess_input, (224, 224)),
    'InceptionV3': (tf.keras.applications.InceptionV3(weights='imagenet', include_top=False, pooling='avg'),
                    tf.keras.applications.inception_v3.preprocess_input, (299, 299)),
    'DenseNet121': (tf.keras.applications.DenseNet121(weights='imagenet', include_top=False, pooling='avg'),
                    tf.keras.applications.densenet.preprocess_input, (224, 224))
}
# Es wird jeweils das vortrainierte Modell mit den imagenet-Gewichten geladen;
# include_top=False bedeutet, dass die oberen Schichten (Klassifikationsschicht) entfernt werden, sodass nur die Convolutional-Layer (Merkmalextraktionsschichten) übrig bleiben
# pooling='avg' fügt eine durchschnittliche Pooling-Schicht hinzu, um den Ausgabe-Feature-Vektor zu reduzieren
# Danach kommt die spezifische Vorverarbeitungsfunktion und die Zielgröße der Bilder


### Extrahieren und Speichern der Feature-Embeddings und Erstellen von Ähnlichkeitsmatrizen ###

# Iteration über das Dictionary mit der zuvor definierten Funktion zur Feature-Extraktion
for model_name, (model, preprocess_input, target_size) in keras_models.items():
    features = extract_features_keras(model, preprocess_input, target_size, ordner_pfad)

    # Dimensionsreduktion mit PCA
    pca = PCA(n_components=2)  # PCA initialisieren
    reduced_features = pca.fit_transform(features)  # PCA anwenden

    # Export der Feature-Embeddings in eine CSV-Datei
    features_df = pd.DataFrame(features)
    features_df.to_csv(f'reduced_embeddings_{model_name}_2.csv', index=False)

    # Berechnen der Ähnlichkeitsmatrix (Berechnet für jeden Merkmalsvektor im Array features die Kosinus-Ähnlichkeit zu jedem anderen Vektor)
    similarity_matrix_2 = cosine_similarity(features)

    # Export der Ähnlichkeitsmatrix in eine CSV-Datei
    df = pd.DataFrame(similarity_matrix_2)
    df.to_csv(f'reduced_similarity_matrix_{model_name}_2.csv', index=False)

    print(f"{model_name} reduced similarity matrix:") # Kann man weglassen, ist nur zum Prüfen des aktuellen Stands und Anschauen der Matrizen
    print(similarity_matrix_2)


### ACHTUNG: Hier wird Kosine-Korrelation verwendet, um zu beurteilen, wie ähnlich zwei Bilder im Merkmalsraum sind, unabhängig von der absoluten Größe der Merkmalsvektoren.


#####################################################################
### Schritt 3: Korrelieren der Ähnlichkeitsmatrizen untereinander ###
#####################################################################

# Wieder Dictionary erstellen zum Zugreifen auf die CSVs
similarity_matrices = {
    'ResNet50': pd.read_csv('reduced_similarity_matrix_ResNet50_2.csv'),
    'VGG16': pd.read_csv('reduced_similarity_matrix_VGG16_2.csv'),
    'InceptionV3': pd.read_csv('reduced_similarity_matrix_InceptionV3_2.csv'),
    'DenseNet121': pd.read_csv('reduced_similarity_matrix_DenseNet121_2.csv')
}

# Korrelation der Ähnlichkeitsmatrizen berechnen
correlations = {}
for name1, mat1 in similarity_matrices.items():
    for name2, mat2 in similarity_matrices.items():
        if name1 < name2:                     # Sicherstellen, dass jedes Modellpaar nur einmal betrachtet wird
            mat2 = mat2.set_index(mat1.index) # Sicherstellen, dass die beiden Ähnlichkeitsmatrizen mat1 und mat2 denselben Index haben
            corr = mat1.stack().corr(mat2.stack())
            correlations[f'{name1} vs {name2}'] = corr

# Ausgabe der Korrelationen
for comparison, correlation in correlations.items():
    print(f"Die Korrelation zwischen {comparison} beträgt: {correlation:.4f}")

# Exportiere die Korrelationsdaten
correlations_df = pd.DataFrame(list(correlations.items()), columns=['Vergleich', 'Korrelation'])
correlations_df.to_csv('reduced_aehnlichkeitsmatrix_korrelationen_2.csv', index=False)

### ACHTUNG: Es wird Pearson-Korrelation verwendet, um die Matrizen zu korrelieren
### (Es wird bewertet, ob hohe (bzw. niedrige) Kosinus-Ähnlichkeitswerte in einer Matrix auch hohe (bzw. niedrige) Werte in der anderen Matrix haben)


'''
######################################################################################
### Schritt 3: a) Berechnen des Durchschnitts menschlicher Ähnlichkeitsratings     ###
###            b) Vergleich zwischen Modellen und menschlicher Durchschnittsmatrix ###
###            c) Vergleich zwischen Modell und einzelnen menschlichen Ratings     ###
###            d) Vergleich der menschlichen Ratings untereinander                 ###
######################################################################################
###                                                                                ###
###    ->  DIE MENSCHENURTEILE GIBT ES BISLANG NOCH NICHT !!!                      ###
###                                                                                ###
### Sobald sie da sind, kann man einen Ordner "Menschenunrteile_2" in Google Colab ###
### erstellen, die Urteile darin hochladen und dann den Code einfach komplett      ###
### laufen lassen.                                                                 ###
###                                                                                ###
######################################################################################


import os
import pandas as pd
import numpy as np
from scipy.stats import pearsonr # Das kann man alles weglassen, wenn der ganze Codenblock auf einmal runt

# Pfad zum Ordner mit den menschlichen Ähnlichkeitsratings
menschen_ordner_pfad = '/content/Menschenurteile_2'
model_ordner_pfad = '/content/'

# Lade menschliche Ähnlichkeitsratings aus dem Ordner
def load_human_ratings(menschen_ordner_pfad):
    dateien = sorted(os.listdir(menschen_ordner_pfad))
    human_ratings = []
    for datei in dateien:
        if datei.endswith('.csv'):
            pfad = os.path.join(menschen_ordner_pfad, datei)
            df = pd.read_csv(pfad)
            human_ratings.append(df.values)  # CSV in ein Numpy-Array umwandeln
    return human_ratings, dateien

# Berechne den Durchschnitt der menschlichen Ähnlichkeitsratings
def average_human_ratings(human_ratings):
    return np.mean(human_ratings, axis=0)

# Lade DNN-Ähnlichkeitsmatrix
def load_model_similarity_matrix(model_name, model_ordner_pfad):
    pfad = os.path.join(model_ordner_pfad, f'similarity_matrix_{model_name}_2.csv')
    return pd.read_csv(pfad).values

# Berechne die Pearson-Korrelation zwischen zwei Matrizen
# Zuerst werden die Matrizen in eindimensionale Arrays umgewandelt, und dann wird die Korrelation berechnet
def calculate_correlation(matrix1, matrix2):
    matrix1_flat = matrix1.flatten()
    matrix2_flat = matrix2.flatten()
    correlation, _ = pearsonr(matrix1_flat, matrix2_flat)
    return correlation

# Erstellen und Speichern der durchschnittlichen menschlichen Matrix in eine CSV-Datei
human_ratings, human_filenames = load_human_ratings(menschen_ordner_pfad)
average_human_matrix = average_human_ratings(human_ratings)
pd.DataFrame(average_human_matrix).to_csv('average_human_ratings_2.csv', index=False)

# Lade die Ähnlichkeitsmatrizen der Modelle
model_names = ['ResNet50', 'VGG16', 'InceptionV3', 'DenseNet121']
model_correlations = {}
human_model_correlations = {}

# Vergleiche jedes Modell mit den durchschnittlichen menschlichen Ähnlichkeitsratings
for model_name in model_names:
    model_matrix = load_model_similarity_matrix(model_name, model_ordner_pfad)

    # Korrelation zwischen Modell und durchschnittlicher menschlicher Matrix
    correlation_avg = calculate_correlation(model_matrix, average_human_matrix)
    model_correlations[model_name] = correlation_avg
    print(f"Korrelation zwischen {model_name} und den durchschnittlichen menschlichen Ratings: {correlation_avg:.4f}") # Aktuellen Stand checken

    # Korrelation zwischen Modell und einzelnen menschlichen Ratings
    for i, human_matrix in enumerate(human_ratings):
        correlation = calculate_correlation(model_matrix, human_matrix)
        human_model_correlations[f'{human_filenames[i]} vs {model_name}'] = correlation
        print(f"Korrelation zwischen {model_name} und {human_filenames[i]}: {correlation:.4f}") # Aktuellen Stand checken

# Vergleiche die menschlichen Ratings untereinander
human_correlations = {}
for i, human_matrix1 in enumerate(human_ratings):
    for j, human_matrix2 in enumerate(human_ratings):
        if i < j:
            correlation = calculate_correlation(human_matrix1, human_matrix2)
            human_correlations[f'Person {i+1} vs Person {j+1}'] = correlation
            print(f"Korrelation zwischen Person {i+1} und Person {j+1}: {correlation:.4f}") # Aktuellen Stand checken

# Speichern der Korrelationen in CSV-Dateien
pd.DataFrame(list(model_correlations.items()), columns=['Modell', 'Korrelation']).to_csv('model_vs_human_average_correlations_2.csv', index=False)
pd.DataFrame(list(human_model_correlations.items()), columns=['Vergleich', 'Korrelation']).to_csv('human_vs_model_correlations_2.csv', index=False)
pd.DataFrame(list(human_correlations.items()), columns=['Vergleich', 'Korrelation']).to_csv('human_correlations_2.csv', index=False)



####################################################################################
### Schritt 5: Tabellarische Gegenüberstellung der einzelnen Ähnlichkeitsratings ###
####################################################################################

# Erstellen einer Liste mit allen Modellnamen und menschlichen Bewertungen (inklusive Average)
all_names = model_names + ['Mensch-Average'] + human_filenames

# Initialisieren einer leeren DataFrame für die Korrelationsmatrix
correlation_table = pd.DataFrame(index=all_names, columns=all_names)

# Vergleiche Modelle untereinander, den Mensch-Average und die einzelnen menschlichen Bewertungen
for name1 in all_names:
    # Lade die entsprechende Ähnlichkeitsmatrix für name1
    if name1 in model_names:
        matrix1 = load_model_similarity_matrix(name1, model_ordner_pfad)
    elif name1 == 'Mensch-Average':
        matrix1 = average_human_matrix
    else:
        matrix1 = human_ratings[human_filenames.index(name1)]

    # Vergleiche name1 mit allen anderen (Modellen, Menschen, Average)
    for name2 in all_names:
        # Lade die entsprechende Ähnlichkeitsmatrix für name2
        if name2 in model_names:
            matrix2 = load_model_similarity_matrix(name2, model_ordner_pfad)
        elif name2 == 'Mensch-Average':
            matrix2 = average_human_matrix
        else:
            matrix2 = human_ratings[human_filenames.index(name2)]

        # Berechne die Korrelation zwischen matrix1 und matrix2
        correlation = calculate_correlation(matrix1, matrix2)

        # Füge die Korrelation in die DataFrame ein
        correlation_table.loc[name1, name2] = correlation

# Speichern der Korrelationsmatrix als CSV-Datei
correlation_table.to_csv('correlation_matrix_table_2.csv', index=True)

print("Task completed")
'''

